In [ ]:
pip install fasttext

     |████████████████████████████████| 68 kB 3.1 MB/s 
  Using cached pybind11-2.7.1-py2.py3-none-any.whl (200 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3096561 sha256=3b43490501809cae20b41c3b4c1d2b6808db1251e268f4c9338ad8334e01e971
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/DCC/data'

/content/drive/MyDrive/DCC/data


In [ ]:
import pandas as pd
import numpy as np
import fasttext
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
#파일 불러오기
train = pd.read_csv('nlp/train.csv', encoding = 'utf-8')
test = pd.read_csv('nlp/test_x.csv', encoding = 'utf-8')
submission = pd.read_csv('nlp/sample_submission.csv', encoding = 'utf-8')

In [ ]:
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)
test['text']=test['text'].apply(alpha_num)

In [ ]:
nltk.download("stopwords")
#불용어 처리
def stop_words(text):
    r = []
    stop_words = list(set(stopwords.words('english')))
    word = text.split(' ')
    for w in word:
      if w not in stop_words:
        r.append(w)    
    return ' '.join(r)

train['text_stopword'] = train['text'].apply(stop_words)
test['text_stopword']=test['text'].apply(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
file = open('./nlp/fasttext_train.txt','w+')
for i in train.index:
    line = '__label__' + str(train['author'][i])+' '+train['text_stopword'][i]
    file.write(line+"\n")

In [ ]:
model = fasttext.train_supervised('./nlp/fasttext_train.txt', epoch=30, minCount=2, maxn=10, verbose=0)

In [22]:
for i in test.index:
    lable, proba = model.predict(test['text_stopword'][i], k=5)
    for lab, pro in zip(lable, proba):
        if '__label__0' == lab:
            submission.loc[i, '0'] = pro
        elif '__label__1' == lab:
            submission.loc[i, '1'] = pro
        elif '__label__2' == lab:
            submission.loc[i, '2'] = pro
        elif '__label__3' == lab:
            submission.loc[i, '3'] = pro
        elif '__label__4' == lab:
            submission.loc[i, '4'] = pro

In [24]:
submission.to_csv('./nlp/submission_fasttext.csv', index=False)